In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
import sys
sys.path.append('../')

from dataset import*
from baselines import*
from synthetic_concept_model import *
from synthetic_coop_model import *
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle
import torch
import numpy
import random
import os

random.seed(7)
numpy.random.seed(seed=7)
torch.manual_seed(7)

In [3]:
# !nvidia-smi
torch.cuda.is_available()

True

## Data generation

In [4]:
feature_dim_info = dict()
label_dim_info = dict()
transform_dim = 100000

intersections = get_intersections(num_modalities=2)

feature_dim_info['12'] = 10
feature_dim_info['1'] = 6
feature_dim_info['2'] = 6

label_dim_info['12'] = 10
label_dim_info['1'] = 6
label_dim_info['2'] = 6
num_concepts = 1
transforms_2concept = None
transforms_2hd = None
num_data = 1000
noise=0.3
pos_prob=0.5
total_data, total_labels, total_concepts, total_raw_features = generate_data_concepts(num_data, num_concepts,
                                                                                      feature_dim_info,
                                                                                      label_dim_info,
                                                                                      transform_dim=transform_dim,
                                                                                     noise=noise,
                                                                                     pos_prob=pos_prob)

# Data splitting & loading

dataset = MultiConcept(total_data, total_labels, total_concepts, 0)
batch_size = 100
trainval_dataset, test_dataset = torch.utils.data.random_split(dataset,  
                                                            [int(0.5 * num_data), num_data - int(0.5 * num_data)])
train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset,
                                                           [int(0.8 * len(trainval_dataset)), len(trainval_dataset) - int(0.8 * len(trainval_dataset))])

Current generated data : 0
Current generated data : 100
Current generated data : 200
Current generated data : 300
Current generated data : 400
Current generated data : 500
Current generated data : 600
Current generated data : 700
Current generated data : 800
Current generated data : 900


In [5]:
# transform_dim = 101024

## Experiment 1


## Scenario 1 
One known concept $c_1$ derived from information components $W_{U_1}, W_s$. Label $Y$ is composed of information components $y=f(W_{U_1}, W_s, W_{U_2})$. We try to recover $W_{U_2}$ by $\arg \max_{Z_x} I(Z_x;Y|Z_{c_1})$, assuming that $Z_{c_1}$ represents $\{W_{U_1}, W_s\}$


In [6]:
"""

!!! CHANGE THE train_concept_encoder AND  train_concept_informed_model FUNCTIONS IF NEEDED!!!

"""

def factorCBM_exp1(train_dataset, test_dataset, concept_encoder, model, device, num_eval=10, save_path='./results'):

    acc_list, pre_list, recall_list, f1_list = [], [], [], []
    # teval = tqdm(range(num_eval))
    for idx in range(num_eval):
        train_loader = DataLoader(train_dataset, shuffle=True, drop_last=True,
                          batch_size=batch_size)
        val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size, drop_last=True)
        test_loader = DataLoader(test_dataset, shuffle=False, drop_last=False)
        # train concpet encoder
        trained_concept_encoder = train_concept_encoder(concept_encoder, train_loader,val_loader, transform_dim, device, 1e-5, 1e-5, 25, 3, '../trained_models')
        # train concept informed model
        trained_concept_informed_model = train_concept_informed_model(trained_concept_encoder, model, train_loader, val_loader, 100, device, 1e-5, 25, 3, '../trained_models')
        
        train_embeds_1 = trained_concept_encoder.get_embedding(torch.stack([sample[0] for sample in  train_dataset]).to(device)).detach().cpu().numpy()
        train_embeds_2 = trained_concept_informed_model.get_embedding(torch.stack([sample[0] for sample in  train_dataset]).to(device)).detach().cpu().numpy()
        train_embeds = np.concatenate((train_embeds_1, train_embeds_2), axis=1) #train_embeds_2 #
        train_labels = np.array([sample[-1].item() for sample in  train_dataset])

        test_embeds_1 = trained_concept_encoder.get_embedding(torch.stack([sample[0] for sample in  test_dataset]).to(device)).detach().cpu().numpy()
        test_embeds_2 = trained_concept_informed_model.get_embedding(torch.stack([sample[0] for sample in  test_dataset]).to(device)).detach().cpu().numpy()
        test_embeds = np.concatenate((test_embeds_1, test_embeds_2), axis=1) #test_embeds_2 #
        test_labels = np.array([sample[-1].item() for sample in  test_dataset])

        clf = LogisticRegression(max_iter=1000).fit(train_embeds, train_labels)
        predictions = clf.predict(test_embeds)
        
        accuracy = accuracy_score(test_labels, predictions)
        precision = precision_score(test_labels, predictions)
        recall = recall_score(test_labels, predictions)
        f1 = f1_score(test_labels, predictions)
        
        acc_list.append(accuracy)
        pre_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
    
    dict_results = {'accuracy':acc_list,
                    'precision':pre_list,
                    'recall':recall_list,
                    'f1_score':f1_list}
    df_results = pd.DataFrame(data=dict_results)
    
    print(f'Accuracy:{df_results.accuracy.mean():0.3f} \u00B1 {2*df_results.accuracy.std():0.3f}')
    print(f'Precision:{df_results.precision.mean():0.3f} \u00B1 {2*df_results.precision.std():0.3f}')
    print(f'Recall:{df_results.recall.mean():0.3f} \u00B1 {2*df_results.recall.std():0.3f}')
    print(f'F1-score:{df_results.f1_score.mean():0.3f} \u00B1 {2*df_results.f1_score.std():0.3f}')
    
    directory = save_path + '/' + time.strftime("%Y%m%d")
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    timestr = time.strftime("%H%M%S")
    file_path = directory +  '/factorCBM_exp1_'+ str(num_eval) + '_' + timestr +'.csv'
    df_results.to_csv(file_path)
    
    
    return df_results

In [7]:
# models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hidden_dim = 512
embed_dim = 50 
concept_encoder = ConceptEncoder(transform_dim, embed_dim, 1, hidden_dim).to(device)
model = ConceptCLSUP_full_concept(transform_dim, embed_dim, 2, hidden_dim, embed_dim).to(device)

In [8]:
results = factorCBM_exp1(train_dataset, test_dataset, concept_encoder, model, device, num_eval=10, save_path='./results')

Epoch 69:   0%|          | 69/100000 [00:18<6:30:16,  4.27it/s, loss=2e-5]    

Val loss did not improve


Epoch 6:   0%|          | 6/100000 [00:01<6:30:10,  4.27it/s, loss=0.000484] 

Val loss did not improve


Epoch 6:   0%|          | 6/100000 [00:01<6:14:54,  4.45it/s, loss=0.000404]

...

Val loss did not improve


Epoch 3:   0%|          | 3/100000 [00:01<11:26:51,  2.43it/s, loss=0.0015]  

Val loss did not improve


Epoch 99: 100%|██████████| 100/100 [00:26<00:00,  3.76it/s, loss=-1.8e+3]


Accuracy:0.850 ± 0.031
Precision:0.813 ± 0.056
Recall:0.901 ± 0.019
F1-score:0.854 ± 0.024


## Scenario 2

Use both the supervised loss and the Info_club constraint to minimize the mutual information between the learned representation and c, then concatenate c to the learned representation and train everything end-to-end.

$\mathop{\arg \min}\limits_{\theta , \phi} \mathcal{L}\bigl( y, f_{\theta}(g_{\phi}(x),c)\bigr) + \lambda Info_{NCE\_CLUB} \bigl( g_{\phi}(x);c \bigr)$


In [9]:
"""

!!! CHANGE THE train_concept_encoder AND  train_concept_informed_model FUNCTIONS IF NEEDED!!!

"""
    
class ConceptCLSUP_Sc2(nn.Module):
    def __init__(self, x_dim, hidden_dim, embed_dim, layers=2, activation='relu', lr=1e-4, concept_dim = 1):
        super(ConceptCLSUP_Sc2, self).__init__()
        self.critic_hidden_dim = 512
        self.critic_layers = 1
        self.critic_activation = 'relu'

        # encoders
        self.backbone = mlp(x_dim, hidden_dim, embed_dim, layers, activation)
        self.linears_infonce = mlp(embed_dim, embed_dim, embed_dim, 1, activation) 
        self.y_projection = mlp(embed_dim + concept_dim, embed_dim, 1, 1, activation)

        # critics
        self.club_critic = CLUBInfoNCECritic(embed_dim + x_dim, concept_dim, self.critic_hidden_dim, self.critic_layers, self.critic_activation)

    def forward(self, x, c):
        # compute embedding
        z = self.linears_infonce(self.backbone(x))
        # compute critic scores
        club_infonce_score = self.club_critic(torch.cat([z, x], dim=-1), c)
        y_encoding = self.y_projection(torch.cat([z,c], dim=-1))
        
        return club_infonce_score, y_encoding

    
    def get_embedding(self, x):
        return self.backbone(x)
    
    def get_logits(self, x, c):
        z = self.linears_infonce(self.backbone(x))
        return self.y_projection(torch.cat([z,c], dim=-1))
    
    def get_backbone(self):
        return self.backbone
    
    
def train_concept_informed_model_sc2(concept_encoder, model, train_loader,val_loader, num_epochs, device, lr, lamb, log_interval,
                          save_interval, save_path):

    # concept_encoder.eval()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    label_loss_func = torch.nn.BCELoss()
    best_val_err = torch.tensor(1e7)
    tepoch = tqdm(range(num_epochs))
    for epoch in tepoch:
        tepoch.set_description(f"Epoch {epoch}")
        model.train()
        for batch_idx, (data, concept, target) in enumerate(train_loader):
            data, concept, target = data.to(device), concept.to(device) ,target.to(device)
            optimizer.zero_grad()
            c, z_c = concept_encoder(data)
            MI_loss, y_logits = model(data, c) #oncept
            label_loss = label_loss_func(torch.sigmoid(y_logits), target.float())
            loss = label_loss + lamb * MI_loss
            loss.backward()
            optimizer.step()

            # torch.cuda.empty_cache()
            # gc.collect()
            
        tepoch.set_postfix(loss=loss.item())
        if epoch % save_interval == 0:
            val_err = 0
            model.eval()
            with torch.no_grad():
                for batch_idx, (data, concept, target) in enumerate(val_loader):
                    data, concept, target = data.to(device), concept.to(device), target.to(device)
                    c , z_c = concept_encoder(data)
                    # output = model(data, c, target) #z_c
                    MI_loss, y_logits = model(data, c)#oncept
                    label_loss = label_loss_func(torch.sigmoid(y_logits), target.float())
                    loss = label_loss + lamb * MI_loss
                    val_err += loss
                val_err = val_err / len(val_loader)
            if val_err < best_val_err:
                best_val_err = val_err

            else:
                print('Val loss did not improve')
                torch.save(model.state_dict(), os.path.join(save_path, 'concept_informed_model.pth'))
                return model
    return model

def factorCBM_exp1_Sc2(train_dataset, test_dataset, concept_encoder, model, device, num_eval=10, save_path='./results'):

    acc_list, pre_list, recall_list, f1_list = [], [], [], []
    # teval = tqdm(range(num_eval))
    for idx in range(num_eval):
        train_loader = DataLoader(train_dataset, shuffle=True, drop_last=True,
                          batch_size=batch_size)
        val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size, drop_last=True)
        test_loader = DataLoader(test_dataset, shuffle=False, drop_last=False)
        # train concpet encoder
        trained_concept_encoder = train_concept_encoder(concept_encoder, train_loader,val_loader, 100, device, 1e-5, 1e-5, 25, 3, '../trained_models')
        # train concept informed model concept_encoder
        trained_concept_informed_model = train_concept_informed_model_sc2(trained_concept_encoder, model, train_loader, val_loader, num_epochs=100, device=device,
                                                                          lr=1e-5, lamb=0.5, log_interval = 25, save_interval = 3, save_path ='../trained_models')
        
        test_embeds =  torch.stack([sample[0] for sample in  test_dataset]).to(device)
        test_concepts = torch.tensor([sample[1] for sample in  test_dataset]).unsqueeze(1).to(device)
        test_labels = np.array([sample[-1].item() for sample in  test_dataset])

        
        out = trained_concept_informed_model.get_logits(test_embeds, test_concepts)
        
        predictions = torch.sigmoid(out).round().detach().cpu().numpy()
        
        accuracy = accuracy_score(test_labels, predictions)
        precision = precision_score(test_labels, predictions)
        recall = recall_score(test_labels, predictions)
        f1 = f1_score(test_labels, predictions)
        
        acc_list.append(accuracy)
        pre_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
    
    dict_results = {'accuracy':acc_list,
                    'precision':pre_list,
                    'recall':recall_list,
                    'f1_score':f1_list}
    df_results = pd.DataFrame(data=dict_results)
    
    print(f'Accuracy:{df_results.accuracy.mean():0.3f} \u00B1 {2*df_results.accuracy.std():0.3f}')
    print(f'Precision:{df_results.precision.mean():0.3f} \u00B1 {2*df_results.precision.std():0.3f}')
    print(f'Recall:{df_results.recall.mean():0.3f} \u00B1 {2*df_results.recall.std():0.3f}')
    print(f'F1-score:{df_results.f1_score.mean():0.3f} \u00B1 {2*df_results.f1_score.std():0.3f}')
    
    directory = save_path + '/' + time.strftime("%Y%m%d")
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    timestr = time.strftime("%H%M%S")
    file_path = directory +  '/factorCBM_exp1_Sc2_'+ str(num_eval) + '_' + timestr +'.csv'
    df_results.to_csv(file_path)
    
    
    return df_results

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hidden_dim = 512
embed_dim = 50
concept_encoder = ConceptEncoder(transform_dim, embed_dim, 1, hidden_dim).to(device)
model = ConceptCLSUP_Sc2(transform_dim, hidden_dim, embed_dim, 2, 'relu').to(device)

In [11]:
results = factorCBM_exp1_Sc2(train_dataset, test_dataset, concept_encoder, model, device, num_eval=10, save_path='./results')

Epoch 78:   0%|          | 78/100000 [00:19<5:48:22,  4.78it/s, loss=1.07e-5] 

Val loss did not improve


Epoch 12:   0%|          | 12/100000 [00:03<6:45:23,  4.11it/s, loss=0.000417]

Val loss did not improve


Epoch 39:  39%|███▉      | 39/100 [00:51<01:17,  1.27s/it, loss=-.0744]

...

Val loss did not improve


Epoch 6:   6%|▌         | 6/100 [00:08<01:58,  1.27s/it, loss=-.137]

Val loss did not improve


Epoch 6:   6%|▌         | 6/100 [00:09<02:35,  1.66s/it, loss=-.137]


Accuracy:0.848 ± 0.021
Precision:0.918 ± 0.038
Recall:0.756 ± 0.082
F1-score:0.828 ± 0.035


## Baseline 1 (logistic regression on $x$)

In [12]:
results_1 = baseline_1(train_dataset, test_dataset, num_eval=10)

Evaluation 9: 100%|██████████| 10/10 [02:45<00:00, 16.57s/it]

Accuracy:0.864 ± 0.000
Precision:0.876 ± 0.000
Recall:0.840 ± 0.000
F1-score:0.858 ± 0.000


## Baseline 2 (Supervised Representation Learning on $x$)

In [13]:
results_2 = baseline_2(train_dataset, val_dataset, test_dataset, transform_dim=transform_dim, batch_size=batch_size, num_eval=10)

Epoch 400:  40%|████      | 400/1000 [01:29<02:13,  4.49it/s, loss=1.97e-5] 


Val loss did not improve


Epoch 900:  90%|█████████ | 900/1000 [03:21<00:22,  4.46it/s, loss=4.59e-6] 


Val loss did not improve


Epoch 900:  90%|█████████ | 900/1000 [03:30<00:23,  4.28it/s, loss=3.41e-6] 


...

Val loss did not improve


Epoch 500:  50%|█████     | 500/1000 [01:58<01:58,  4.23it/s, loss=8.68e-6] 


Val loss did not improve


Epoch 700:  70%|███████   | 700/1000 [02:45<01:10,  4.24it/s, loss=2.75e-6] 

Val loss did not improve
Accuracy:0.865 ± 0.011
Precision:0.887 ± 0.019
Recall:0.829 ± 0.044
F1-score:0.857 ± 0.016


## Baseline 3 

### (Logistic Regression on $x,c_1$)

In [14]:
resuts_3_A = baseline_3_A(train_dataset, test_dataset, num_eval=10, save_path='./results')

Evaluation 9: 100%|██████████| 10/10 [02:44<00:00, 16.50s/it]

Accuracy:0.870 ± 0.000
Precision:0.884 ± 0.000
Recall:0.844 ± 0.000
F1-score:0.864 ± 0.000


### (Supervised Representation Learning on $x,c_1$)

In [15]:
results_3_b = baseline_3_B(train_dataset, val_dataset, test_dataset, transform_dim=transform_dim, batch_size=batch_size, num_eval=10)

Epoch 600:  60%|██████    | 600/1000 [02:18<01:32,  4.34it/s, loss=5.88e-6] 


Val loss did not improve


Epoch 800:  80%|████████  | 800/1000 [03:06<00:46,  4.29it/s, loss=2.17e-6] 


Val loss did not improve


Epoch 700:  70%|███████   | 700/1000 [02:42<01:09,  4.31it/s, loss=5.75e-6] 


...

Val loss did not improve


Epoch 800:  80%|████████  | 800/1000 [03:07<00:46,  4.26it/s, loss=2.76e-6] 


Val loss did not improve


Epoch 999: 100%|██████████| 1000/1000 [03:52<00:00,  4.29it/s, loss=3.24e-6]


Accuracy:0.865 ± 0.007
Precision:0.885 ± 0.022
Recall:0.832 ± 0.035
F1-score:0.857 ± 0.010


## Baseline 4 (Multi-Task Learning with Concepts $x \rightarrow y, c_1$)

In [16]:
results_4 = baseline_4(train_dataset, val_dataset, test_dataset, transform_dim=transform_dim, batch_size=batch_size, num_eval=10)

Epoch 600:  60%|██████    | 600/1000 [02:15<01:30,  4.43it/s, loss=0.000589]


Val loss did not improve


Epoch 900:  90%|█████████ | 900/1000 [03:31<00:23,  4.26it/s, loss=5.18e-5] 


Val loss did not improve


Epoch 500:  50%|█████     | 500/1000 [25:39<25:39,  3.08s/it, loss=0.00305]   

...

Val loss did not improve


Epoch 500:  50%|█████     | 500/1000 [04:16<04:16,  1.95it/s, loss=0.00124] 

Val loss did not improve



Epoch 800:  80%|████████  | 800/1000 [30:34<07:38,  2.29s/it, loss=0.000284]  


Val loss did not improve
Accuracy:0.866 ± 0.007
Precision:0.888 ± 0.016
Recall:0.830 ± 0.011
F1-score:0.858 ± 0.007


## Baseline 5 (Pre-Training with Concepts $x \rightarrow c_1, x \rightarrow y$)

In [18]:
results_5 = baseline_5(train_dataset, val_dataset, test_dataset,w5=0, transform_dim=transform_dim, batch_size=batch_size, num_eval=10)

Epoch 500:  50%|█████     | 500/1000 [01:58<01:58,  4.23it/s, loss=0.000129]


Val loss did not improve


Epoch 900:  90%|█████████ | 900/1000 [03:40<00:24,  4.07it/s, loss=3.7e-6]  


Val loss did not improve


Epoch 500:  50%|█████     | 500/1000 [02:06<02:06,  3.94it/s, loss=0.000329]


...

Val loss did not improve


Epoch 500:  50%|█████     | 500/1000 [02:07<02:07,  3.92it/s, loss=8.47e-5] 


Val loss did not improve


Epoch 600:  60%|██████    | 600/1000 [02:30<01:40,  3.98it/s, loss=8.89e-6] 

Val loss did not improve
Accuracy:0.865 ± 0.018
Precision:0.882 ± 0.071
Recall:0.839 ± 0.136
F1-score:0.857 ± 0.034


## Plots

In [ ]:
from analysis import *

In [ ]:
get_w5_curves(baseline= 'factorCBM_exp1', w5_list=['1', '2', '3','5', '7', '10', '12'])

In [ ]:
get_w5_curves(baseline= 'baseline_2', w5_list=['1', '2', '3','5', '7', '10', '12'])

In [ ]:
get_w5_curves(baseline= 'baseline_3_A', w5_list=['1', '2', '3','5', '7', '10', '12'])

In [ ]:
get_w5_curves(baseline= 'baseline_3_B', w5_list=['1', '2','3','5', '7', '10', '12'])

In [ ]:
get_w5_curves(baseline= 'baseline_4', w5_list=['1','3','5', '10', '12'])

In [ ]:
get_w5_curves(baseline= 'baseline_5', w5_list=['1', '3','5', '10', '12'])